# Week 3 Part 3

#### As maps are not visible on GitHub, here's the link to the same notebook on IBM Watson: [Notebook](https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/a3f0304b-cecb-42f6-89bb-8b3415ba71bb/view?access_token=0db9e1ecb5f3988199221d5940b3e748e140e49e0f2fef4c303c10c6638684aa)



In [1]:
import numpy as np
import pandas as pd
#pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import folium
print("Libraries imported.")

Libraries imported.


## Load the saved data from part 2

In [2]:
toronto_df = pd.read_csv('toronto_neigh_latlong.csv')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Use geopy to obtain lat,long of Toronto

In [3]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Generate map of Toronto with the Neighborhoods

In [4]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map

## Initiate FourSquare API Credentials

In [5]:
CLIENT_ID = 'FRUG1QP2AWAUDZ3VPAHTTYPTCI1SE2PDABPIXDZZ22N4MVII' # your Foursquare ID
CLIENT_SECRET = 'IWZMB4QOMBYPWEP4JFXENLBL3FX2LBRE2FQMBWFJOQYOMWAJ' # your Foursquare Secret
VERSION = '20190801'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FRUG1QP2AWAUDZ3VPAHTTYPTCI1SE2PDABPIXDZZ22N4MVII
CLIENT_SECRET:IWZMB4QOMBYPWEP4JFXENLBL3FX2LBRE2FQMBWFJOQYOMWAJ


## Query the API to obtain 100 venues within 500m of each Postal Code

In [6]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['PostalCode'], toronto_df['Borough'], toronto_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2244, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [7]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M1B,Scarborough,"Rouge, Malvern",1,1,1,1,1,1
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
M1E,Scarborough,"Guildwood, Morningside, West Hill",7,7,7,7,7,7
M1G,Scarborough,Woburn,3,3,3,3,3,3
M1H,Scarborough,Cedarbrae,7,7,7,7,7,7
M1J,Scarborough,Scarborough Village,3,3,3,3,3,3
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3


In [8]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
venues_df['VenueCategory'].unique()[:25]

There are 280 uniques categories.


array(['Fast Food Restaurant', 'Bar', 'Pizza Place', 'Electronics Store',
       'Mexican Restaurant', 'Rental Car Location', 'Medical Center',
       'Intersection', 'Breakfast Spot', 'Coffee Shop',
       'Korean Restaurant', 'Hakka Restaurant', 'Caribbean Restaurant',
       'Thai Restaurant', 'Athletics & Sports', 'Bank', 'Bakery',
       'Fried Chicken Joint', 'Playground', 'Spa', 'Convenience Store',
       'Department Store', 'Discount Store', 'Chinese Restaurant',
       'Bus Station'], dtype=object)

## Breakdown each Area by Category

In [9]:
toronto_catdf = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_catdf['PostalCode'] = venues_df['PostalCode'] 
toronto_catdf['Borough'] = venues_df['Borough'] 
toronto_catdf['Neighborhoods'] = venues_df['Neighborhood'] 
toronto_catdf['Lat'] = venues_df['BoroughLatitude']
toronto_catdf['Long'] = venues_df['BoroughLongitude']

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_catdf.columns[-5:]) + list(toronto_catdf.columns[:-5])
fixed_columns
toronto_catdf = toronto_catdf[fixed_columns]

print(toronto_catdf.shape)
toronto_catdf.head()

(2244, 285)


,PostalCode,Borough,Neighborhoods,Lat,Long,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Summarize each Area by Category

In [10]:
toronto_grouped = toronto_catdf.groupby(["PostalCode", "Borough", "Neighborhoods", "Lat", "Long"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(101, 285)


,PostalCode,Borough,Neighborhoods,Lat,Long,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Get the Top 10 Categories for each Area

In [11]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods', "Lat", "Long"]
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']
neighborhoods_venues_sorted['Lat'] = toronto_grouped['Lat']
neighborhoods_venues_sorted['Long'] = toronto_grouped['Long']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[5:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 5:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(101, 15)


,PostalCode,Borough,Neighborhoods,Lat,Long,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Dive Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Breakfast Spot,Rental Car Location,Intersection,Pizza Place,Electronics Store,Medical Center,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Cuban Restaurant,Costume Shop,Farmers Market


## Cluster Areas

In [12]:
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods", "Lat", "Long"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [13]:
neighborhoods_clust = neighborhoods_venues_sorted.copy()

# add clustering labels
neighborhoods_clust["ClusterLabel"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

fixed_columns = list(neighborhoods_clust.columns[-1:]) + list(neighborhoods_clust.columns[:-1])
fixed_columns
neighborhoods_clust = neighborhoods_clust[fixed_columns]

print(neighborhoods_clust.shape)
# print(neighborhoods_clust.groupby('ClusterLabel').count()['PostalCode'])
neighborhoods_clust.head()

(101, 16)


,ClusterLabel,PostalCode,Borough,Neighborhoods,Lat,Long,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Dive Bar
2,1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Breakfast Spot,Rental Car Location,Intersection,Pizza Place,Electronics Store,Medical Center,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
3,1,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Cuban Restaurant,Costume Shop,Farmers Market


## Map the Clusters

In [14]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(neighborhoods_clust['Lat'], neighborhoods_clust['Long'], neighborhoods_clust['PostalCode'], neighborhoods_clust['Borough'], neighborhoods_clust['Neighborhoods'], neighborhoods_clust['ClusterLabel']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyze the Clusters

In [15]:
neighborhoods_clust.loc[neighborhoods_clust['ClusterLabel'] == 0].head()

,ClusterLabel,PostalCode,Borough,Neighborhoods,Lat,Long,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,0,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,Park,Playground,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
20,0,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,Park,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
22,0,M2P,North York,York Mills West,43.752758,-79.400049,Park,Convenience Store,Bank,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
24,0,M3A,North York,Parkwoods,43.753259,-79.329656,Food & Drink Shop,Park,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Discount Store
29,0,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,Park,Airport,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


In [16]:
neighborhoods_clust.loc[neighborhoods_clust['ClusterLabel'] == 1].head()

,ClusterLabel,PostalCode,Borough,Neighborhoods,Lat,Long,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Dive Bar
2,1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Breakfast Spot,Rental Car Location,Intersection,Pizza Place,Electronics Store,Medical Center,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
3,1,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Cuban Restaurant,Costume Shop,Farmers Market
